# Teste 4 - API

Para este teste, primeiramente como pedido foi criado um servidor usando o <code>Flask</code> do Python para construir nossa API que devolve os resultados da busca, e posteriomente, foi criado nosso cliente usando a framework Vue.js para a realização da busca em si pelo usuário final.

O código do <code>app.py</code> que emulará nosso servidor Flask com a logica da pesquisa pode ser conferido a seguir:

In [ ]:
from flask import Flask, jsonify, request
import mysql.connector as mysql

app = Flask(__name__)

conn = mysql.connect(
    host='localhost',
    user='root',
    password='',
    database='psintutivecare')
    
cursor = conn.cursor()

def check_query(query):
    if not query:
        return jsonify([{"erro" : "O parametro query está vazio."}])
    return query

def execute_query(query):
    sql_query = f"SELECT * FROM operadoras_ativas \
        WHERE registro_ans LIKE '%{query}%' \
        OR cnpj LIKE '%{query}%' \
        OR razao_social LIKE '%{query}%' \
        OR nome_fantasia LIKE '%{query}%'"

    cursor.execute(sql_query)
    return cursor.fetchall()

def jsonify_result(results):
    if not results:
        return jsonify([{"erro" : "Nenhum resultado encontrado."}])
    row_headers = [h[0] for h in cursor.description]
    json_data = []
    for result in results:
        json_data.append(dict(zip(row_headers,result)))
    return jsonify(json_data)

@app.route('/buscar', methods=['GET'])
def buscar():
    query = request.args.get('query')
    query = check_query(query)
    results = execute_query(query)
    
    response = jsonify_result(results)
    response.headers.add("Access-Control-Allow-Origin", "*")
    
    return response

Algumas considerações:

- O Flask conectou-se diretamente com o MySQL para a maninupação dos dados.
- Para realizar a busca deve-se bater no end-point <code>/busca?query={termo buscado}</code>
- É retornado um <code>json</code> com os resultados.
- É buscado semelhanças do termo desejado com os campos Registro ANS, CNPJ, Razão Social e Nome fantasia.
- Foi adicionado o header <code>Access-Control-Allow-Origin</code> para conseguirmos acessar a API de outras fontes (nosso cliente).
   

<img src="imagens/json_api.png" width="600" height="600">

Em seguida, foi criado nosso cliente Buscador utilizando o Vue.js, onde os principais arquivos foram o <code>main.js</code> e o <code>App.vue</code>.

In [ ]:
import { createApp } from 'vue'
import App from './App.vue'

export const eventBus = createApp(App)

createApp(App).mount('#app')

In [ ]:
<template>
  <div class="container">
    <div class="jumbotron d-flex flex-column mx-4 my-4">
      <h1 class="display-4">Operadoras</h1>
      <p class="lead">Busque as informações sobre as operadoras ativas aqui.</p>
      <div class="form-group mb-3">
        <label for="query">Buscar:</label>
        <input type="text" v-model="query" id="query" class="form-control" placeholder="Digite a busca">
        <button @click="search" class="btn btn-primary mx-2 my-2">Pesquisar</button>
      </div>
    </div>
    <div class="table-container mx-4 my-4">
      <h3 class="display-6">Resultados:</h3>
      <div class="table-responsive">
        <table class="table table-striped">
          <thead>
            <tr>
              <th v-for="header in headers" :key="header">{{ header }}</th>
            </tr>
          </thead>
          <tbody>
            <tr v-for="result in results" :key="result.id">
              <td v-for="header in headers" :key="header">{{ result[header] }}</td>
            </tr>
          </tbody>
        </table>
      </div>
    </div>
  </div>
</template>

<script>
const axios = require('axios');

export default {
  data() {
    return {
      query: '',
      results: [],
      headers: []
    }
  },
  methods: {
    async search() {
      const response = await axios.get(`http://localhost:5000/buscar?query=${this.query}`)
      console.log(response.data)
      this.results = response.data
      this.headers = Object.keys(this.results[0])
    }
  },
}
</script>

Os servidores foram colocados em uma instância EC2 da Amazon para teste e a url temporária foi enviada junto ao teste, como resultado final obtemos a seguinte visualização por parte do Cliente:

<img src="imagens/screen-cliente.gif" width="700" height="900">

Todos esses códigos na estrutura correta podem ser encontrados na pasta <code>teste4-API</code>.